# FT105 - MLP
## Lucas Domingos Monteiro
## RA 272387

### Environment setup

In [ ]:
!pip install ucimlrepo

import pandas as pd
import matplotlib.pyplot as mpl
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

### Data preprocessing
I decided that the model would have a single hidden layer, due to the relatively small-sized dataset. To decide on the number of neurons of the layer, I used a rule-of-thumb extracted from https://medium.com/geekculture/introduction-to-neural-network-2f8b8221fbd3

In [ ]:
# fetch dataset
liver_disorders = fetch_ucirepo(id=60)

# data (as pandas dataframes)
features = liver_disorders.data.features
targets = liver_disorders.data.targets

# number for the single hidden neuron layer
hidden_neuron_n = int(((2/3) * len(features.columns)) + len(targets.columns))

# converting targets to categorical, then to numerical using one-hot-encoding
targets = pd.DataFrame({'drinks': pd.cut(targets['drinks'], bins=3, labels=['Few', 'Some', 'Plenty'])})
encoder = OneHotEncoder(sparse_output=False, dtype=int)
targets = encoder.fit_transform(targets)

### Subsampling, classification and scoring
While splitting the datasets, the `stratify` parameter was removed since there was only 1 member of the least populated class in the target set. This yielded an error while trying to use automatic stratification.

For the `MLPClassifier`, "LBFGS" was chosen as the solver because as advised in [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier), it might perform better and converge faster, for small datasets.

Also for the classifier, the `alpha` parameter was not provided as to use the default of `0.0001`. It controls the strength of the L2 regularization term.

Here, the data was scaled to a 0-1 interval using the `StandardScaler` object.
The scaling is first performed on the training set, then on the test set with the same parameters.

In [ ]:
# table for final presentation
final_data = pd.DataFrame(columns=["Accuracy"])

# Random subsampling - 5 repetitions
for iteration in range(5):

  # split into 30/70 sets
  f_train, f_test, t_train, t_test = train_test_split(features, targets, test_size=0.3, random_state=iteration)

  # feature scalling
  scaler = StandardScaler()
  scaler.fit(f_train)
  f_train = scaler.transform(f_train)
  f_test = scaler.transform(f_test) # applies the same scaling from the training sent onto the test set

  # create classifier, fit, and predict
  clf = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes=(hidden_neuron_n,), max_iter=1000, random_state=iteration)
  clf.fit(f_train, t_train)
  pred = clf.predict(f_test)

  # plot accuracy to final table
  accuracy = accuracy_score(t_test, pred)
  final_data.loc[len(final_data)] = accuracy

### Final comparison
With the previous approach (ensemble) performing with an mean accuracy of `0.8384615384615385`, the results are nearly identical.

I reckon that this serves as a statement to the precision of a ensemble, in which traditional models can perform very similar to an MLP when combined.

It is also a statement to the synthetic power of MLPs, which single-handedly achieves the power of 3 traditional methods combined, even though its inner processings might be a bit more convoluted.

In [ ]:
print("Mean error rate:", float(final_data.mean()))
final_data

Mean error rate: 0.8038461538461539


,Accuracy
0,0.788462
1,0.798077
2,0.778846
3,0.836538
4,0.817308
